In [15]:

import openai
from openai import OpenAI
import os
import re
from dotenv import load_dotenv
import json
import google.generativeai as genai
import os
import time
import sqlite3
import logging
import mysql.connector
from collections import Counter
import math
from sentence_transformers import SentenceTransformer, util

In [8]:
def getDataBase(path):
    with open(path) as f:
        data = json.load(f)
    return data

In [10]:
def getSchema(path):


    with open(path) as f:
        data = json.load(f)
    return data

In [6]:
# Get the current working directory
script_dir = os.getcwd()

# Construct absolute paths to the files
dataBasePath = os.path.join(script_dir, "../../../ATIS/combined.json")

# Schema Path 
schemaPath = os.path.join(script_dir, "../schemaCreator/combined_schema.json")

In [11]:

data = getDataBase(dataBasePath)
schema = getSchema(schemaPath)

In [14]:

script_dir = os.getcwd()

logPath = os.path.join(script_dir, "../logs/correctGemini.log")
inCorrectlogPath = os.path.join(script_dir, "../logs/incorrectGemini.log")

with open(logPath, 'w'):
    pass   
with open(inCorrectlogPath, 'w'):
    pass


In [16]:
def get_first_row_with_columns(database_path, table_name):
    try:
        conn = sqlite3.connect(database_path)
        cursor = conn.cursor()
        
        # Get column names
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = [info[1] for info in cursor.fetchall()]
        
        # Get the first row
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 1")
        first_row = cursor.fetchone()
        
        conn.close()
        
        return columns, first_row
    except sqlite3.OperationalError as e:
        print(f"An error occurred: {e}")
        with open('./incorrectGeminiLog.txt', 'a') as f:
            f.write(f"\nError that occurred:\n{e}\n\n")
        return None, None

def execute_query(database_path, query):
    try:
        conn = sqlite3.connect(database_path)
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        conn.close()
        return results
    except sqlite3.OperationalError as e:
        print(f"An error occurred: {e}")
        with open('./incorrectGeminiLog.txt', 'a') as f:
            f.write(f"\nError that occured:\n{e}\n\n")
        return None

def table_desc_extractor(table_info, table_description):
    table_extract = {}
    for table in table_info:
        table_name = table[0]
        table_col = str(table[1])
        table_row = str(table[2])
        # Check if table_name exists in table_description and is not None, else use a default value
        table_desc = table_description.get(table_name, "No description") if table_description.get(table_name) is not None else "No description"
        table_extract[table_name] ={
            "table_description": table_desc,
            "table_columns": table_col,
            "table_first_row": table_row
        }
    return table_extract

def extract_sql_query(response):
    start = response.find("```sqlite")
    flag=0
    if (start==-1):
        flag=1
        start = response.find("```sql")
    end = response.find("```", start + 1)
    if start == -1 or end == -1:
        return ""
    sql_query=""
    if flag==1: sql_query = response[start + 6:end].strip()  # Extract the query between ```sql and ```
    else: sql_query = response[start + 9:end].strip()
    return sql_query

def convert_to_list(results):
    final_list = []
    if not results: return final_list
    for row in results:
        final_list.append(tuple(sorted(list(row), key=str)))
    return final_list

def compare_queries(db_name, db_folder_path, generated_query, original_query):
    db_path = os.path.join(db_folder_path, db_name, db_name + '.sqlite')
    if not os.path.exists(db_path):
        print(f"Database {db_name} not found.")
        return False

    generated_results = execute_query(db_path, generated_query)
    original_results = execute_query(db_path, original_query)

    # print(generated_results)
    # print(original_results)

    gen_list = convert_to_list(generated_results)
    orig_list = convert_to_list(original_results)

    print(gen_list)
    print(orig_list)

def extract_table_descriptions(response):
    # Define the pattern to extract table_name and table_description
    pattern = r"# Table (\w+)\s*\n(.+?)\n"
    matches = re.findall(pattern, response, re.DOTALL)

    # Create a dictionary {table_name: table_description}
    table_extraction_dict = {match[0]: match[1].strip() for match in matches}
    
    return table_extraction_dict